In [2]:
import numpy as np
from PIL import Image
import sys


def curve(pixel):
    r, g, b = pixel
    brightness = r + g + b
    if brightness < 60:
        k = 60 / brightness
        return min(255, int(r * k ** 2)), min(255, int(g * k ** 2)), min(255, int(b* k
** 2))
    else:
        return r, g, b


im = Image.open("image.png")
pixels = im.load() # список с пикселями
x, y = im.size # ширина (x) и высота (y) изображения
print(x, y)
n = min(x,y)
for i in range(n):
    for j in range(i+1, n):
        r, g, b = pixels[i, j]
        pixels[i, j], pixels[j, i] =  pixels[j, i], pixels[i, j]
im.save("Риана2.jpg")






# Загружаем исходное изображение и преобразуем в RGB
img = Image.open("image.png").convert("RGB")
data = np.array(img)
h, w, channels = data.shape

# Выполняем 2D-преобразование Фурье для каждого цветового канала
fft_data = np.zeros((h, w, channels), dtype=np.complex128)
for ch in range(channels):
    fft_data[:, :, ch] = np.fft.fft2(data[:, :, ch])

# Вычисляем логарифмический масштаб модуля (для лучшей визуализации)
magnitude = np.abs(fft_data)
magnitude_log = np.log1p(magnitude)

# Нормализуем каждую компоненту до диапазона [0,255]
magnitude_norm = np.zeros_like(magnitude_log)
for ch in range(channels):
    max_val = magnitude_log[:, :, ch].max()
    if max_val > 0:
        magnitude_norm[:, :, ch] = 255 * magnitude_log[:, :, ch] / max_val

# Округляем до целых чисел и приводим к типу uint8
magnitude_img = np.rint(magnitude_norm).astype(np.uint8)


Image.fromarray(magnitude_img, mode="RGB").save("out.jpg")
print("Fourier образ сохранён в файле out.jpg")

filtered_fft = np.copy(fft_data)
filtered_fft[h//2:, :, :] = 0  
filtered_fft[:, w//2:, :] = 0  

# Выполняем обратное преобразование Фурье для каждого канала
filtered_img = np.zeros((h, w, channels), dtype=np.float64)
for ch in range(channels):
    inv_channel = np.fft.ifft2(filtered_fft[:, :, ch])
    filtered_img[:, :, ch] = np.real(inv_channel)

# Ограничиваем диапазон значений до [0, 255] и округляем до целых чисел
filtered_img = np.clip(filtered_img, 0, 255)
filtered_img = np.rint(filtered_img).astype(np.uint8)


Image.fromarray(filtered_img, mode="RGB").save("out2.jpg")
print("Обратное преобразованное изображение сохранено в файле out2.jpg")




800 600
Fourier образ сохранён в файле out.jpg
Обратное преобразованное изображение сохранено в файле out2.jpg


# Задание 2. Фурье-преобразование звука.

Возьмите файл in10.wav и, используя sound.py, добавьте в его фурье-образ низкочастотный шум. Сохраните результат в out1.wav. После этого обрежьте нижние частоты в фурье-образе и сохраните в out2.wav.